In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 954 kB 7.8 MB/s 
     |████████████████████████████████| 138 kB 59.7 MB/s 
     |████████████████████████████████| 356 kB 52.0 MB/s 
     |████████████████████████████████| 3.5 MB 22.7 MB/s 
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
from selenium import webdriver
import matplotlib.pyplot as plt
import urllib.request as webInfo
import requests
from google.colab import files
from google.colab import files
from bs4 import BeautifulSoup
import time
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
driver =webdriver.Chrome('chromedriver',options=chrome_options)

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
start = dt.datetime(2020, 9, 1)
end = dt.datetime.now().date()
end_diff_format = end.strftime('%m/%d/%Y')

watchList = ["FB", "HTZGQ", "AAPL"]
watching = 0

ttm_ROE = 0;
sector_var = ""
industry_var = ""

# 0 - unclear, 1 - buy, 2 - sell
globalResultsDict = {}
globalResultsDict["4-year Increasing Total Revenue"] = 0
globalResultsDict["4-year Increasing Gross Profit"] = 0
globalResultsDict["4-year Decreasing Operating Expense"] = 0
globalResultsDict["4-year Increasing Operating Income"] = 0
globalResultsDict["4-year Operating Income Rising Faster than Operating Expenses"] = 0
globalResultsDict["4-year Increasing Basic EPS"] = 0
globalResultsDict["4-year Increasing Normalized Income"] = 0
globalResultsDict["4-year Increasing EBIT"] = 0
globalResultsDict["Better EBITDA / Sales Ratio as compared to competitors"] = 0

In [ ]:
# Get S&P500
sp500 = pd.read_csv("/content/constituents_csv.csv")
sp500_symbol = sp500["Symbol"]
sp500_symbol

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object

In [ ]:
def df_add_based_on_ticker(ticker):
  # URL link 
  financials = 'https://finance.yahoo.com/quote/' + ticker + '/financials?p=' + ticker
  balanceSheet = 'https://finance.yahoo.com/quote/' + ticker + '/balance-sheet?p=' + ticker
  cashFlow = 'https://finance.yahoo.com/quote/' + ticker + '/cash-flow?p=' + watchList[watching]
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ FINANCIALS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ FINANCIALS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ FINANCIALS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

  class AppURLopener(webInfo.FancyURLopener):
      version = "Mozilla/5.0"
  opener = AppURLopener()
  response = opener.open(financials)


  read_data = response.read() 
  soup_is= BeautifulSoup(read_data,'lxml')

  features = soup_is.find_all('div', class_='D(tbr)')
  headers = []
  temp_list = []
  label_list = []
  final = []
  index = 0
  #create headers
  for item in features[0].find_all('div', class_='D(ib)'):
      headers.append(item.text)
  #statement contents
  while index <= len(features)-1:
      #filter for each line of the statement
      temp = features[index].find_all('div', class_='D(tbc)')
      for line in temp:
          #each item adding to a temporary list
          temp_list.append(line.text)
      #temp_list added to final list
      final.append(temp_list)
      #clear temp_list
      temp_list = []
      index+=1
  df = pd.DataFrame(final[1:])
  df.columns = headers
  df = df[:-5]
  df = df.T

  new_header = df.iloc[0]
  df = df[1:] 
  df.columns = new_header

  df_1 = df.rename(columns = {'Ticker': 'Year'}, inplace = False)

  df_1.index.name = '' # Remove the index name
  df_1.rename(index={'ttm': end_diff_format},inplace=True) #Rename ttm in index columns to end of the year
  df_1 = df_1.reset_index()
  df_1 = df_1.rename(columns = {'': 'Year'}, inplace = False)
  df_1["Year"] = df_1["Year"].apply(lambda x: dt.datetime.strptime(str(x), '%m/%d/%Y'))
  df_1["Year"] = df_1["Year"].apply(lambda x: x.year)

  second_column = []
  for i in range(len(df_1)):
    second_column.append('Financials')
  
  df_1.insert(0, 'Financials', second_column)

  first_column = []
  for i in range(len(df_1)):
    first_column.append(ticker) 
  
  df_1.insert(0, 'Ticker', first_column)

  #df_1.set_index('Ticker', inplace=True)

  tempdf = df_1.T
  new_header = tempdf.iloc[0]
  tempdf = tempdf[1:] 
  tempdf.columns = new_header

  new_new_df = tempdf.T
  new_new_df

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
# MEGA FINANCIAL CSV

listdeleter = [g for g in sp500_symbol]
each_df = []

counter = 0
while (len(listdeleter) != 0 and counter < 499):
  for i in listdeleter:
    try:
      calculated_df = df_add_based_on_ticker(i)
      each_df.append(calculated_df)
    except:
      time.sleep(120)
      print("YAHOO HATES U")
      continue

    listdeleter.pop(listdeleter.index(i))
    print("{}/500, ticker: {} done".format(counter, i))
    counter+=1


In [ ]:
req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

def news_df_create(ticker, req_headers):
    news = 'https://www.marketwatch.com/investing/stock/' + ticker
    with requests.Session() as s:
        url = news
        r = s.get(url, headers=req_headers)
    soup_is = BeautifulSoup(r.content, 'html.parser')

    news_list = []
    for roe in soup_is.find_all('div', attrs={'class': 'collection__elements j-scrollElement'}):
        for ro_1 in roe.find_all('div'):
            for a in ro_1.find_all('a', attrs={'class': 'figure__image'}):

                img = a.find('img', alt=True)
                if (img is not None and img['alt'] == 'Read full story'):
                    news_list.append(a['href'])

    # Headlines
    headlines = []
    relevant_h = news_list[0:5];
    for article in relevant_h:
        with requests.Session() as s:
            url = article
            r = s.get(url, headers=req_headers)
        soup_is = BeautifulSoup(r.content, 'html.parser')

        for headline in soup_is.find_all('h1', attrs={'class': 'article__headline'}):
            headlines.append(headline.text)

    # DF
    news_df = pd.DataFrame()
    news_df["Ticker"] = [ticker] * len(headlines)
    news_df["Headline"] = headlines
    news_df["Article Link"] = relevant_h
    #news_df["Positivity Score"] = np.array(1)

    news_df["Headline"] = news_df["Headline"].apply(lambda x: x.replace('\n', ""))
    news_df["Article Link"] = news_df["Article Link"]

    return news_df

In [ ]:
news_df = news_df_create('AAPL', req_headers) 
news_df

,Ticker,Headline,Article Link
0,AAPL,Opinion: What to know before making your holid...,https://www.marketwatch.com/story/what-to-know...
1,AAPL,Democrats plan revamp for green cards in pos...,https://www.marketwatch.com/story/democrats-pl...
2,AAPL,U.S. Companies Face New Risks in a Changing ...,https://www.marketwatch.com/articles/us-china-...
3,AAPL,"Apple Inc. stock rises Friday, outperforms m...",https://www.marketwatch.com/story/apple-inc-st...
4,AAPL,AMC Begins Accepting Bitcoin and Ethereum as...,https://www.marketwatch.com/articles/amc-bitco...


In [ ]:
# MEGA FINANCIAL CSV

listdeleter = [g for g in sp500_symbol]
each_df = []

counter = 0
while (len(listdeleter) != 0 and counter < 499):
  for i in listdeleter:
    try:
      calculated_df = news_df_create(i, req_headers)
      each_df.append(calculated_df)
    except:
      time.sleep(120)
      print("YAHOO HATES U")
      continue

    listdeleter.pop(listdeleter.index(i))
    print("{}/500, ticker: {} done".format(counter, i))
    counter+=1

0/500, ticker: MMM done
1/500, ticker: ABT done
2/500, ticker: ABMD done
3/500, ticker: ATVI done
4/500, ticker: ADBE done
5/500, ticker: AMD done
6/500, ticker: AFL done
7/500, ticker: APD done
8/500, ticker: ALK done
9/500, ticker: ARE done
10/500, ticker: ALLE done
11/500, ticker: ALL done
12/500, ticker: GOOG done
13/500, ticker: AMZN done
14/500, ticker: AEE done
15/500, ticker: AEP done
16/500, ticker: AIG done
17/500, ticker: AWK done
18/500, ticker: ABC done
19/500, ticker: AMGN done
20/500, ticker: ADI done
21/500, ticker: ANTM done
22/500, ticker: APA done
23/500, ticker: AMAT done
24/500, ticker: ANET done
25/500, ticker: AIZ done
26/500, ticker: ATO done
27/500, ticker: ADP done
28/500, ticker: AVB done
29/500, ticker: BKR done
30/500, ticker: BAC done
31/500, ticker: BAX done
32/500, ticker: BRK.B done
33/500, ticker: BIO done
34/500, ticker: BIIB done
35/500, ticker: BK done
36/500, ticker: BKNG done
37/500, ticker: BXP done
38/500, ticker: BMY done
39/500, ticker: BR don

In [ ]:
def insider_df_creation(ticker, req_headers):
    insiders = 'http://openinsider.com/screener?s={}&o=&pl=&ph=&ll=&lh=&fd=730&fdr=&td=0&tdr=&fdlyl=&fdlyh=&daysago=&xp=1&xs=1&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=100&page=1'.format(
        ticker)
    with requests.Session() as s:
        url = insiders
        try:
            r = s.get(url, headers=req_headers)
        except:
            time.sleep(5)
            r = s.get(url, headers=req_headers)

    soup_is = BeautifulSoup(r.content, 'html.parser')
    insider_columns = ['Filing_Date', 'Trade Date', 'Ticker', 'Insider_Name', 'Title', 'Trade Type', 'Price', 'Qty',
                       'Owned', 'Own', 'Value']

    insider_df = pd.DataFrame(columns=insider_columns)
    skip = 0
    for trades in soup_is.find_all('table', attrs={'class': 'tinytable'}):
        for trs in trades.find_all('tr'):
            curr_list = []
            for tds in trs.find_all('td'):
                if (len(tds.text) > 1):
                    curr_list.append(tds.text)
            if (len(curr_list) == len(insider_columns)):
                insider_df.loc[len(insider_df)] = curr_list

    indices_to_remove = []
    for idx, i in insider_df.iterrows():
        if (str(i['Ticker']).replace(' ', '') != ticker):
            indices_to_remove.append(idx)

    insider_df = insider_df.drop(indices_to_remove)
    insider_df["Trade Date"] = insider_df["Trade Date"].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
    insider_df_curr_year = insider_df[insider_df["Trade Date"].apply(lambda x: x.year == dt.datetime.now().year)]
    if (insider_df_curr_year.empty):
        return pd.DataFrame()
    insider_df_curr_year['Qty'] = insider_df_curr_year['Qty'].apply(lambda x: round(float(x.replace(',', '')), 2))
    insider_df_curr_year.loc[insider_df_curr_year['Qty'] <= 0, 'Type'] = 'Sale'
    insider_df_curr_year.loc[insider_df_curr_year['Qty'] > 0, 'Type'] = 'Buy'
    insider_df_curr_year_small = insider_df_curr_year[
        ["Filing_Date", "Trade Date", "Insider_Name", "Title", "Type", "Price", "Qty", "Owned", "Own", "Value"]]
    insider_df_curr_year_small = insider_df_curr_year_small[0:10]

    insider_df_curr_year_small["Trade Date"] = insider_df_curr_year_small["Trade Date"].apply(lambda x: "{} {}".format(x.strftime('%B')[0:3], x.year))


    insider_df_curr_year_small.insert(0, 'Ticker', [ticker] * len(insider_df_curr_year_small))
    insider_df_curr_year_small = insider_df_curr_year_small.drop(['Trade Date', 'Own'], axis=1, inplace=False)

    insider_df_curr_year_small["Price"] = insider_df_curr_year_small["Price"].apply(lambda x: float(x.split('$')[1].replace(',', '')))
    insider_df_curr_year_small["Value"] = insider_df_curr_year_small["Value"].apply(lambda x: int(x.split('$')[1].replace(',', '')))
    insider_df_curr_year_small["Owned"] = insider_df_curr_year_small["Owned"].apply(lambda x: int(x.replace(',', '')))

    return insider_df_curr_year_small

In [ ]:
insider_df = insider_df_creation('AAPL', req_headers)
insider_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,Ticker,Filing_Date,Insider_Name,Title,Type,Price,Qty,Owned,Value
0,AAPL,2021-11-03 18:31:28,Adams Katherine L.,"SVP, GC, Secretary",Sale,148.68,-25000.0,447993,3716895
1,AAPL,2021-10-21 18:30:48,Maestri Luca,"SVP, CFO",Sale,148.62,-165829.0,110501,24645506
2,AAPL,2021-10-19 18:31:45,O'Brien Deirdre,SVP,Sale,146.35,-18247.0,136118,2670391
3,AAPL,2021-10-05 18:44:42,Williams Jeffrey E,COO,Sale,138.83,-165829.0,489490,23022040
4,AAPL,2021-10-05 18:43:32,O'Brien Deirdre,SVP,Sale,138.83,-89437.0,136118,12416539
5,AAPL,2021-08-26 18:30:46,Cook Timothy D,CEO,Sale,148.58,-2386440.0,3279726,354568479
6,AAPL,2021-08-09 18:30:51,O'Brien Deirdre,SVP,Sale,145.99,-14877.0,136118,2171893
7,AAPL,2021-08-04 18:30:53,Adams Katherine L.,"SVP, GC, Secretary",Sale,145.89,-17000.0,307164,2480048
8,AAPL,2021-05-05 18:31:41,Maestri Luca,"SVP, CFO",Sale,132.81,-121072.0,110501,16079338
9,AAPL,2021-05-05 18:30:33,Adams Katherine L.,"SVP, GC, Secretary",Sale,132.80,-17000.0,328174,2257631


In [ ]:
listdeleter = [g for g in sp500_symbol]
each_df = []

counter = 0
while (len(listdeleter) != 0 and counter < 499):
  for i in listdeleter:
    try:
      calculated_df = insider_df_creation(i, req_headers)
      each_df.append(calculated_df)
    except:
      time.sleep(120)
      print("YAHOO HATES U")
      continue

    listdeleter.pop(listdeleter.index(i))
    print("{}/500, ticker: {} done".format(counter, i))
    counter+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

0/500, ticker: MMM done


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

1/500, ticker: ABT done


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

498/500, ticker: BDX done
499/500, ticker: FRT done


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

500/500, ticker: MDLZ done
501/500, ticker: TSCO done


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [ ]:
result_f = pd.concat(each_df)
result_f = result_f.set_index('Ticker')
result_f

,Filing_Date,Insider_Name,Title,Type,Price,Qty,Owned,Value
Ticker,,,,,,,,
MMM,2021-08-12 11:51:40,Lavers Jeffrey R,Group Pres,Sale,200.00,-6025.0,6453,1205000
MMM,2021-08-11 10:05:40,Banovetz John Patrick,EVP,Sale,197.93,-3236.0,10513,640498
MMM,2021-08-10 10:02:13,Ashish Khandpur K,Group Pres,Sale,196.86,-6505.0,21958,1280582
MMM,2021-08-04 10:43:47,Hammes Eric D.,Group Pres,Sale,201.83,-1765.0,6131,356145
MMM,2021-08-03 09:59:39,Rutherford Denise R,SVP,Sale,198.17,-3135.0,10812,621270
...,...,...,...,...,...,...,...,...
TSCO,2021-05-12 19:05:09,Korzekwa Christi C,SVP Marketing,Sale,200.04,-8836.0,7980,1767507
TSCO,2021-05-05 17:24:35,Yankee Colin,EVP Supply Chain,Sale,189.94,-3586.0,9590,681132
TSCO,2021-04-22 17:03:50,Mills Robert D,EVP CTO,Sale,185.00,-5892.0,18611,1090020


In [ ]:
def etf_exposure_create(ticker, req_headers):
    chief_ticker = ticker
    etf_exposure = 'https://etfdb.com/stock/{}/'.format(ticker)
    with requests.Session() as s:
        url = etf_exposure
        r = s.get(url, headers=req_headers)
    soup_is = BeautifulSoup(r.content, 'html.parser')

    ticker_list = []
    etf_name_list = []
    category_list = []
    expense_ratio_list = []
    weightage_list = []
    for etf_finder in soup_is.find_all('table', attrs={
        'class': 'table mm-mobile-table table-module2 table-default table-striped table-hover table-pagination'}):
        for tbody in etf_finder.find_all('tbody'):
            for tr in tbody.find_all('tr'):
                if (tr.find("td", attrs={'data-th': 'Ticker'}).a is not None):
                    ticker = tr.find("td", attrs={'data-th': 'Ticker'}).a.text
                else:
                    ticker = tr.find("td", attrs={'data-th': 'Ticker'}).text
                if (tr.find("td", attrs={'data-th': 'ETF'}).a is not None):
                    etf_name = tr.find("td", attrs={'data-th': 'ETF'}).a.text
                else:
                    etf_name = tr.find("td", attrs={'data-th': 'ETF'}).text
                if (tr.find("td", attrs={'data-th': 'ETF Database Category'}).a is not None):
                    category = tr.find("td", attrs={'data-th': 'ETF Database Category'}).a.text
                else:
                    category = tr.find("td", attrs={'data-th': 'ETF Database Category'}).text
                if (tr.find("td", attrs={'data-th': 'Expense Ratio'}).a is not None):
                    expense_ratio = tr.find("td", attrs={'data-th': 'Expense Ratio'}).a.text
                else:
                    expense_ratio = tr.find("td", attrs={'data-th': 'Expense Ratio'}).text
                if (tr.find("td", attrs={'data-th': 'Weighting'}).a is not None):
                    weightage = tr.find("td", attrs={'data-th': 'Weighting'}).a.text
                else:
                    weightage = tr.find("td", attrs={'data-th': 'Weighting'}).text
                ticker_list.append(ticker)
                etf_name_list.append(etf_name)
                category_list.append(category)
                expense_ratio_list.append(expense_ratio)
                weightage_list.append(weightage)

    ETF_exposure_df = pd.DataFrame()
    ETF_exposure_df['ETF_Ticker'] = ticker_list
    ETF_exposure_df['ETF_Name'] = etf_name_list
    ETF_exposure_df['Category'] = category_list
    ETF_exposure_df['Expense_Ratio'] = expense_ratio_list
    ETF_exposure_df['Weightage'] = weightage_list

    ETF_exposure_df_10 = ETF_exposure_df[0:10]

    ETF_exposure_df_10['Expense_Ratio'] = ETF_exposure_df_10['Expense_Ratio'].apply(lambda x: float(x.split('%')[0]))
    ETF_exposure_df_10['Weightage'] = ETF_exposure_df_10['Weightage'].apply(lambda x: float(x.split('%')[0]))

    ETF_exposure_df_10.insert(0, 'Ticker', [chief_ticker] * len(ETF_exposure_df_10))

    return ETF_exposure_df_10

In [ ]:
etf_df = etf_exposure_create('AAPL', req_headers)
etf_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Ticker,ETF_Ticker,ETF_Name,Category,Expense_Ratio,Weightage
0,AAPL,XLK,Technology Select Sector SPDR Fund,Technology Equities,0.12,20.90
1,AAPL,VGT,Vanguard Information Technology ETF,Technology Equities,0.10,19.98
2,AAPL,FTEC,Fidelity MSCI Information Technology Index ETF,Technology Equities,0.08,18.81
3,AAPL,IXN,iShares Global Tech ETF,Technology Equities,0.43,16.68
4,AAPL,HLAL,Wahed FTSE USA Shariah ETF,Large Cap Growth Equities,0.50,16.59
5,AAPL,IYW,iShares U.S. Technology ETF,Technology Equities,0.43,15.68
6,AAPL,MGK,Vanguard Mega Cap Growth ETF,Large Cap Growth Equities,0.07,12.56
7,AAPL,IWY,iShares Russell Top 200 Growth ETF,Large Cap Growth Equities,0.20,12.25
8,AAPL,IOO,iShares Global 100 ETF,Large Cap Growth Equities,0.40,12.12
9,AAPL,IETC,iShares Evolved U.S. Technology ETF,Technology Equities,0.18,11.72


In [ ]:
listdeleter = [g for g in sp500_symbol]
each_df = []

counter = 0
while (len(listdeleter) != 0 and counter < 499):
  for i in listdeleter:
    try:
      calculated_df = etf_exposure_create(i, req_headers)
      each_df.append(calculated_df)
    except:
      time.sleep(600)
      print("YAHOO HATES U")
      continue

    listdeleter.pop(listdeleter.index(i))
    print("{}/500, ticker: {} done".format(counter, i))
    counter+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0/500, ticker: MMM done
1/500, ticker: ABT done
2/500, ticker: ABMD done
3/500, ticker: ATVI done
4/500, ticker: ADBE done
5/500, ticker: AMD done
6/500, ticker: AFL done
7/500, ticker: APD done
8/500, ticker: ALK done
9/500, ticker: ARE done
10/500, ticker: ALLE done
11/500, ticker: ALL done
12/500, ticker: GOOG done
13/500, ticker: AMZN done
14/500, ticker: AEE done
15/500, ticker: AEP done
16/500, ticker: AIG done
17/500, ticker: AWK done
18/500, ticker: ABC done
19/500, ticker: AMGN done
20/500, ticker: ADI done
21/500, ticker: ANTM done
22/500, ticker: APA done
23/500, ticker: AMAT done
24/500, ticker: ANET done
25/500, ticker: AIZ done
26/500, ticker: ATO done
27/500, ticker: ADP done
28/500, ticker: AVB done
29/500, ticker: BKR done
30/500, ticker: BAC done
31/500, ticker: BAX done
32/500, ticker: BRK.B done
33/500, ticker: BIO done
34/500, ticker: BIIB done
35/500, ticker: BK done
36/500, ticker: BKNG done
37/500, ticker: BXP done
38/500, ticker: BMY done
39/500, ticker: BR don

In [ ]:
result_f = pd.concat(each_df)
result_f = result_f.set_index('Ticker')
result_f

,ETF_Ticker,ETF_Name,Category,Expense_Ratio,Weightage
Ticker,,,,,
MMM,DJD,Invesco Dow Jones Industrial Average Dividend ETF,Large Cap Blend Equities,0.07,4.44
MMM,SCHD,Schwab US Dividend Equity ETF,Large Cap Blend Equities,0.06,3.47
MMM,DIA,SPDR Dow Jones Industrial Average ETF Trust,Large Cap Growth Equities,0.16,3.33
MMM,XLI,Industrial Select Sector SPDR Fund,Industrials Equities,0.12,3.29
MMM,VSDA,VictoryShares Dividend Accelerator ETF,Large Cap Growth Equities,0.35,3.17
...,...,...,...,...,...
UNP,VIS,Vanguard Industrials ETF,Industrials Equities,0.10,3.41
UNP,PAVE,Global X US Infrastructure Development ETF,Utilities Equities,0.47,2.73
UNP,IYJ,iShares U.S. Industrials ETF,Industrials Equities,0.42,2.72


In [ ]:
# Exporting FINANCIALS as csv

result_f.to_csv('insider_df.csv') 
files.download('insider_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>